In [7]:
import pandas as pd
import pyvista as pv
import numpy as np
import ipywidgets as widgets


file_path = "Data/ManeuverPlan.csv"
df = pd.read_csv(file_path)
df = df.dropna(subset=['WaypointX', 'WaypointY', 'WaypointZ'])
object_positions = np.column_stack((df['WaypointX'], df['WaypointY'], df['WaypointZ']))
max_steps = len(object_positions) - 1

def plot_maneuver(step, prev_step_range=0, future_step_range=0):
    plotter = pv.Plotter(notebook=True)
    current_radius = 50
    past_radius = 30
    future_radius = 30
    target_radius = 60

    #sphere at the center that represents what we're rendevous'ing with
    sphere = pv.Sphere(radius=target_radius, center=(0, 0, 0))
    plotter.add_mesh(sphere, color="orange", opacity=0.6)
    

    #sphere to show current position
    x,y,z = object_positions[step]
    current_position = object_positions[step]
    sphere = pv.Sphere(radius=current_radius, center=(x, y, z))
    plotter.add_mesh(sphere, color="blue", opacity=0.6)

    #gather past maneuvers
    if prev_step_range > 0:
        add_mesh=False
        past_combined = pv.PolyData()
        start_step = step - prev_step_range
        start_step = start_step if start_step >= 0 else 0
        past_positions = object_positions[start_step:step]
        for x, y, z in past_positions:
            add_mesh=True
            sphere = pv.Sphere(radius=past_radius, center=(x, y, z))
            past_combined = past_combined.merge(sphere)
        if add_mesh:
            plotter.add_mesh(past_combined, color="gray", opacity=0.6)
            #lines to connect path
            past_path = np.vstack((past_positions, current_position))
            lines = pv.lines_from_points(past_path)
            plotter.add_mesh(lines, color="gray", line_width=2)

    #gather future maneuvers
    if future_step_range > 0:
        add_mesh=False
        future_combined = pv.PolyData()
        end_step = step + future_step_range
        end_step = end_step if end_step <= max_steps else max_steps
        future_positions = object_positions[step+1:end_step+1]
        for x, y, z in future_positions:
            add_mesh=True
            sphere = pv.Sphere(radius=future_radius, center=(x, y, z))
            future_combined = future_combined.merge(sphere)
        if add_mesh:
            plotter.add_mesh(future_combined, color="green", opacity=0.6)
            #lines to connect path
            future_path = np.vstack((current_position, future_positions))
            lines = pv.lines_from_points(future_path)
            plotter.add_mesh(lines, color="green", line_width=2)
    
    plotter.add_axes()
    plotter.show()
    
    

maneuver_select = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1.0,
    description='Manuever Number',
    continuous_update=False
)

previous_maneuver_select = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1.0,
    description='# of Past Maneuvers',
    continuous_update=False
)

future_maneuver_select = widgets.IntSlider(
    value=0,
    min=0,
    max=max_steps,
    step=1.0,
    description='# of Future Maneuvers',
    continuous_update=False
)

widgets.interact(plot_maneuver, step=maneuver_select, prev_step_range=previous_maneuver_select, future_step_range=future_maneuver_select)


interactive(children=(IntSlider(value=0, continuous_update=False, description='Manuever Number', max=43), IntS…

<function __main__.plot_maneuver(step, prev_step_range=0, future_step_range=0)>